# How do I write files to my own _cloud storage_?
### Overview
Connecting to cloud storage facilitates sharing and leveraging bioinformatics data that is commonly stored there (details [here](http://docs.sevenbridges.com/docs/connecting-cloud-storage-overview)). Here, we show an example of **writing** files from the _Seven Bridges Platform_ to a users _Amazon Web Services_ **bucket**. 

### Prerequisites
 1. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 2. You need to have an AWS account with an S3 bucket
 3. You need to be a member (or owner) of a project that has some files in it, e.g. if you followed this [recipe](files_copyFromPublicReference.ipynb)
 
### Setting up Identity and Access Management (IAM) access 
There is a detailed [tutorial](http://docs.sevenbridges.com/v1.0/docs/aws-cloud-storage-tutorial) to set up access to your buckets. Please follow that tutorials and make note of your **policy name(s)**. Also make sure to generate an _secret access key_.

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below. We also define<sup>1</sup> a function to wrap the Volumes API calls.

<sup>1</sup> Support for _Volumes API_ is on the immediate roadmap for the _sevenbridges-python_ library, once it is implemented the api_call() function is unnecessary.

In [ ]:
import sevenbridges as sbg

import json
from requests import request


def api_call(path, method='GET', query=None, data=None, token=None):
    
    base_url = 'https://api.sbgenomics.com/v2/storage/'
    
    data = json.dumps(data) if isinstance(data, dict) \
    or isinstance(data,list) else None
              
    headers = {
        'X-SBG-Auth-Token': token,
        'Accept': 'application/json',
        'Content-type': 'application/json',
    }
    
    response = request(method, base_url + path, params=query, \
                       data=data, headers=headers) 
    response_dict = json.loads(response.content) if \
    response.content else {}
    
    if response.status_code / 100 != 2:
        print(response_dict['message'])
        print('Error Code: %i.' % (response_dict['code']))
        print(response_dict['more_info'])
        raise Exception('Server responded with status code %s.' \
                        % response.status_code)
    return response_dict

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Mount the volume
Since we have configured the IAM policy in the prior step, we can now mount the volume.

#### Additional information
 * Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/create-a-volume-v2)

In [ ]:
body = {
  "name": "my_volume",
  "service": {
    "type": "s3",
    "bucket": "BUCKET-NAME",
    "prefix": "",
    "credentials": {
      "access_key_id": "INSERT ACCESS KEY ID",
      "secret_access_key": "INSERT SECRET ACCESS KEY"
    },
    "properties": {
      "sse_algorithm": "AES256"
    }
  },
  "access_mode": "RW"
}

response = api_call(path = 'volumes/', method = 'POST', \
                    token = config_file.auth_token, data = body)

## Get details of the volume
Let's list all our volumes and get one volume's details to make sure it's mounted as we expected.

### Warning
We've hardcoded the single_volume to detail, this could easily be selected by matching 'name' instead

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here (list)](http://docs.sevenbridges.com/docs/list-volumes-v2) and [here (details)](http://docs.sevenbridges.com/docs/get-details-of-a-volume-v2)

In [ ]:
my_volumes = api_call(path = 'volumes', method = 'GET', \
                    token = config_file.auth_token)
for v in my_volumes['items']:
    print(v)
    

single_volume = api_call(path = ('volumes/' + my_volumes['items'][0]['id']), \
                         method = 'GET', token = config_file.auth_token)
print(single_volume)

## Get some files to move
Here we pick a project and a few files to move. In case you don't have any files, you could add them via the [GUI](http://docs.sevenbridges.com/docs/copy-files-using-the-visual-interface) or an API [recipe](files_copyFromPublicReference.ipynb). 

#### PROTIPS
 * A detailed _recipe_ for listing projects is [here](projects_listAll.ipynb)
 * A detailed _recipe_ for listing files is [here](files_listAll.ipynb)

In [ ]:
# [USER INPUT] Set project name here:
project_name = 'Tweaking'
extension = 'cxb'


# Find your project
my_project = [p for p in api.projects.query(limit=100).all() \
             if p.name == project_name]

if my_project:    # exploit fact that empty list is False, {list, tuple, etc} is True
    print('No project named (%s) found, please check spelling' % project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]
    
# List files in that project matching your extension
my_files = api.files.query(limit=100, project=my_project)
selected_files = [f for f in my_files.all() \
                  if f.name[-len(extension):] == extension]

## Start an export job
Spin up a job to export files from the _Seven Bridges Platform_ to your _Cloud Storage_ (Volume).

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](http://docs.sevenbridges.com/docs/start-an-export-job-v2)

In [ ]:
# Loop through selected files, start one job for each.
for f in selected_files[1:]:
    body = {
        "source": {
            "file": f.id
        },
        "destination": {
            "volume": single_volume['id'],
            "location": f.name
        }
    } 
    
    response = api_call(path = 'exports', method = 'POST', \
                    token = config_file.auth_token, data = body)
    print("File (%s) queued for export to your cloud storage" % \
         (f.name))

## List and detail export jobs
The first call (list) is sufficient to keep track of the job progress in 

```python
    export_jobs['items'][0]['state']
```

The second call reveals more detail about the job.

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](http://docs.sevenbridges.com/docs/list-export-jobs-v2) and [here](http://docs.sevenbridges.com/docs/get-details-of-an-export-job-v2)

In [ ]:
export_jobs = api_call(path = 'exports', method = 'GET', \
                    token = config_file.auth_token)

print("Example of information available in the list call:")
print(export_jobs['items'][0])

job_details = api_call(path = ('exports/' + export_jobs['items'][0]['id']), \
                      method = 'GET', token = config_file.auth_token)

print("\n Example of information available in the detail call:")
print(job_details)